=============== 

In [ ]:
path = 'e:'

In [ ]:
import os
os.listdir(path)

### x; vital data 

In [ ]:
# data
import os
path21 = 'e:/2021_1_1_10_14/충남대병원/마취통증의학과/OTHER'

flist = [] 
for path, dirs, files in os.walk(path21):
    for f in files:
        flist.append(os.path.join(path,f))

import os
path21_0 = 'e:/2021_1015_1231/충남대병원/마취통증의학과/OTHER'

flist0 = [] 
for path, dirs, files in os.walk(path21_0):
    for f in files:
        flist0.append(os.path.join(path,f))

path22 = 'e:\\2022\\충남대병원\\마취통증의학과\\OTHER'
flist1 = [] 
for path, dirs, files in os.walk(path22):
    for f in files:
        flist1.append(os.path.join(path,f))

allf = []
allf.extend(flist)
allf.extend(flist0)
allf.extend(flist1)

# remove non-vital files
allf = [x for x in allf if os.path.splitext(allf[0])[-1] == '.vital']

allf_n = [os.path.splitext(os.path.basename(x))[0] for x in allf]

allf_s = [x.split('_') for x in allf_n]

# len(flist),len(flist0),len(flist1)
# (22634, 9046, 9291)

In [ ]:
import pickle 
with open('vital_paths.pkl','wb') as f:
    pickle.dump(allf,f)
with open('vital_fname.pkl','wb') as f:
    pickle.dump(allf_n,f)
with open('vital_split.pkl','wb') as f:
    pickle.dump(allf_s,f)

### y; label data (op list) 

In [ ]:
# label
label_path0 = './회복간호_21상.xlsx'
label_path1 = './회복간호_21하.xlsx'
label_path2= './회복간호_22상.xlsx'

# remove nrs-non-existing rows
# select columns
def get_data(path,filter_='simple'):
    df = pd.read_excel(path,header=1)
    df = df.iloc[df[['최대 NRS']].dropna().index]
    if filter_ == 'simple':
        df = df[['등록번호','회복실퇴실일시','최대 NRS']]
    elif filter_ == 'all':
        pass
    return df

In [ ]:
df0 = get_data(label_path0)
df1 = get_data(label_path1)
df2 = get_data(label_path2)

df = pd.concat([df0,df1,df2])

df = df.drop_duplicates()

In [ ]:
# op_list.csv (filter1)
df0_a = get_data(label_path0,filter_='all')
df1_a = get_data(label_path1,filter_='all')
df2_a = get_data(label_path2,filter_='all')

df_a = pd.concat([df0_a,df1_a,df2_a])

df_a = df_a.drop_duplicates(subset=['등록번호','회복실퇴실일시'])

df_a.to_csv('filter0.csv',index=False)

### matching VitalFiles and LabelFiles 

#### __filter0_simple.csv__: ['등록번호','date','time','최대 NRS'] from filter0.csv 

In [ ]:
import pandas as pd
df = pd.read_csv('filter0.csv')

df = df[['등록번호','회복실퇴실일시','최대 NRS']]

# 2022-01-21 12:13:00 > 210121, 1213

df = df.reset_index(drop=True)
def get_date(date:str,type_='ymd'):
    
    date = date.split()

    y,m,d = date[0].split('-')
    h,min_   = date[1].split(':')
    
    out = ''
    if type_ == 'ymd':
        ymd = y[-2:]+m+d
        out = ymd
    elif type_ == 'hm':
        hm = h+min_
        out = hm
    return out

df['time'] = df['회복실퇴실일시'].map(lambda x: get_date(x,'hm') )

df['date'] = df['회복실퇴실일시'].map(lambda x: get_date(x,'ymd') )
# change the cols. sequence 
df = df[['등록번호','date','time','최대 NRS']]

# len(df0),len(df1),len(df2)
# (6338, 6855, 5508)
# len(df) 18701 (w/ duplicates)

# df.to_csv('filter0_simple.csv',index=False)

#### Matching: find the vitalFile path for each item 

In [52]:
# prepare the materials
import pandas as pd 
df = pd.read_csv('filter0_simple.csv')

import pickle 
with open('vital_paths.pkl','rb') as f:
    allf = pickle.load(f)
with open('vital_fname.pkl','rb') as f:
    allf_n = pickle.load(f)
with open('vital_split.pkl','rb') as f:
    allf_s = pickle.load(f)

In [ ]:
# filter0 ~ vital > (pre&post-existing only) 
idx = []
for j, sample in enumerate(df[['등록번호','date','time']].values): # sample: ['등록번호','date','time','최대 NRS']
    
    sample = sample.astype('str')
                           
                           
    for i, data in enumerate(allf_s): # data: ['00000824', 'ORC14', '210611', '133500', '1']

        if data[0].endswith(sample[0]): # 등록번호
            if data[2].startswith(sample[1]): # 회복실퇴실일시(date) 210611
                idx.append([j, sample,allf_n[i],allf[i]])

dfi = pd.DataFrame(idx,columns = ['idx','label','vital','path'])
# len(dfi) 33198

import pickle 
with open('dfi.pkl','wb') as f:
    pickle.dump(dfi,f)

In [ ]:
import pickle 
with open('dfi.pkl', 'rb') as f:
    dfi = pickle.load(f)

In [ ]:
import os
def extract_path(path,paths=['e:/2021_1_1_10_14/충남대병원/마취통증의학과/OTHER','e:/2021_1015_1231/충남대병원/마취통증의학과/OTHER','e:\\2022\\충남대병원\\마취통증의학과\\OTHER']):
    ext_path = ''
    
    for p in paths:
        if os.path.commonpath([path,p]) == os.path.abspath(p):
            ext_path = os.path.relpath(path,start=p)
            break
    return ext_path

ext_path = dfi.path.map(extract_path)

import pandas as pd
dfi['ext_path'] = ext_path

dfi.to_csv('dfi_1.csv',index=False)
# len(dfi) 20747

In [ ]:
pd.set_option('display.max_columns', 100)

pd.set_option('display.max_rows', 15000)

#### filter; pre-post pair 

In [ ]:
import pandas as pd
dfi = pd.read_csv('./dfi_1.csv')

dfi = dfi[dfi.idx.duplicated(keep=False)]

dfi.to_csv('dfi_2dup.csv',index=False)

In [ ]:
import pandas as pd
dfi = pd.read_csv('dfi_2dup.csv')

In [ ]:
import re
pa = re.compile('\D+')

indices = []
for i in dfi.idx.unique():
    duplicates = dfi[dfi.idx == i]
    flist = [ s.split('_')[1].lower() for s in duplicates['vital'].to_list()] # ['orc8', 'orc8', 'orc8']
    flist = [pa.findall(f)[0] for f in flist]
    
    if ('pdor' in flist) and ('pdrec' in flist):
        indices.append(i)

dfip = dfi[dfi.idx.isin(indices)]

def ext_room(name,num=False):
    room_n = name.split('_')[1]
    if num:
        return room_n
    else:
        room = pa.findall(room_n)[0].lower()
        return room
dfip['room'] = dfip.vital.map(ext_room)

In [ ]:
dfip.to_csv('dfi_3pair.csv',index=False)

### Move necessary files from raw dir to root_dir

In [47]:
import shutil
root_dir = 'D:\pd_gy'

for value in dfip.values: # (0, 'idx'),(1, 'label'),(2, 'vital'),(3, 'path'),(4, 'ext_path'),(5, 'room')"
    src= value[3]
    room = value[5]
    
    if room == 'pdor':
        dst = os.path.join(root_dir,'pdor')
        shutil.copy2(src, dst)
    elif room == 'pdrec':
        dst = os.path.join(root_dir,'pdrec')
        shutil.copy2(src, dst)
    else:
        print("room is not found")

### json file 

json 
{
key: 
pt_id:
date:
time: 
vital_name: 01405184_PDREC02_210329_133000_1
or_path:['./pdor/~.vital',...]
rec_path:['./pdrec/~.vital',...]
nrs:
},
{
key: 
pt_id:
date:
time: 
vital_name: 01405184_PDREC02_210329_133000_1
or_path:['./pdor/~.vital',...]
rec_path:['./pdrec/~.vital',...]
nrs:
}


In [42]:
a = '''
(0, 'idx'),
 (1, 'label'),
 (2, 'vital'),
 (3, 'path'),
 (4, 'ext_path'),
 (5, 'room')
 '''

In [44]:
a.replace('\n','')

"(0, 'idx'), (1, 'label'), (2, 'vital'), (3, 'path'), (4, 'ext_path'), (5, 'room') "

In [40]:
list(zip(list(range(len(dfip.columns))),dfip.columns))

[(0, 'idx'),
 (1, 'label'),
 (2, 'vital'),
 (3, 'path'),
 (4, 'ext_path'),
 (5, 'room')]

In [ ]:
pd.set_option('display.max_rows',500)

In [ ]:
dfi.iloc[:100]

In [ ]:
i = 1
dfi[dfi.idx.duplicated(keep=False)][['idx','label','vital','ext_path']].iloc[100*i:100*(i+1)]

In [ ]:
len(dfi[dfi.idx.duplicated(keep=False)])

In [ ]:
dfi[dfi.idx.duplicated(keep=False)]